# チャットモデルのファインチューニング用のデータ準備と分析

このノートブックは、チャットモデルのファインチューニングに使用されるチャットデータセットを前処理および分析するためのツールとして機能します。
フォーマットエラーのチェック、基本統計の提供、およびファインチューニング費用のトークン数の推定を行います。
ここで示される方法は、babbage-002 や davinci-002 などのモデルに対する[レガシーファインチューニング](https://platform.openai.com/docs/guides/legacy-fine-tuning)に対応しています。
GPT-3.5-turboのファインチューニングについては、[現行のファインチューニングページ](https://platform.openai.com/docs/guides/fine-tuning)を参照してください。

In [1]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

## データの読み込み

最初に、チャットのデータセットを[サンプルのJSONLファイル](https://github.com/openai/openai-cookbook/blob/main/examples/data/toy_chat_fine_tuning.jsonl)から読み込みます。

In [2]:
data_path = "data/toy_chat_fine_tuning.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 5
First example:
{'role': 'system', 'content': 'You are a happy assistant that puts a positive spin on everything.'}
{'role': 'user', 'content': 'I fell off my bike today.'}
{'role': 'assistant', 'content': "It's great that you're getting exercise outdoors!"}


## フォーマットの検証

データセット内の各会話がファインチューニングAPIが期待するフォーマットに従っているかを確認するために、さまざまなエラーチェックを実行できます。エラーは、デバッグが簡単に行えるように、その性質に基づいてカテゴリー分けされています。

1. **データタイプチェック**: データセット内の各エントリが辞書（`dict`）であるかどうかをチェックします。エラータイプ：`data_type`。
2. **メッセージリストの存在確認**: 各エントリに`messages`リストが存在するかどうかをチェックします。エラータイプ：`missing_messages_list`。
3. **メッセージキーチェック**: `messages`リスト内の各メッセージに`role`と`content`のキーが含まれているかどうかを確認します。エラータイプ：`message_missing_key`。
4. **メッセージ内の未認識キー**: メッセージに`role`、`content`、および`name`以外のキーがある場合にログを出力します。エラータイプ：`message_unrecognized_key`。
5. **ロールの検証**: `role`が"system"、"user"、または"assistant"のいずれであるかを確認します。エラータイプ：`unrecognized_role`。
6. **コンテンツの検証**: `content`にテキストデータがあり、文字列であることを確認します。エラータイプ：`missing_content`。
7. **アシスタントメッセージの存在確認**: 各会話に少なくとも1つのアシスタントからのメッセージがあるかどうかを確認します。エラータイプ：`example_missing_assistant_message`。

以下のコードはこれらのチェックを実行し、見つかった各タイプのエラーの数を出力します。これはデバッグや、データセットが次のステップに進む準備ができているかを確認するのに役立ちます。

In [3]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


## トークン数カウントユーティリティ

このノートブックの残りの部分で使用するために、いくつかの便利なユーティリティを定義しましょう。

In [4]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

## データの警告とトークン数

軽量な分析を行うことで、データセットにおける潜在的な問題、例えばメッセージの欠落やメッセージとトークンの数に関する統計的な洞察を提供できます。

1. **システム/ユーザーメッセージの不足**: "system" または "user" メッセージが欠落している対話の数をカウントします。このようなメッセージは、アシスタントの振る舞いを定義し、対話を開始するために重要です。
2. **各例のメッセージ数**: 各対話内のメッセージ数の分布を要約し、対話の複雑さに関する洞察を提供します。
3. **各例の合計トークン数**: 各対話内のトークンの総数を計算し、その分布を要約します。ファインチューニングのコストを理解するために重要です。
4. **アシスタントのメッセージ内のトークン数**: 各対話ごとにアシスタントのメッセージ内のトークン数を計算し、この分布を要約します。アシスタントの冗長性を理解するのに役立ちます。
5. **トークン制限の警告**: 例が最大トークン制限（4096トークン）を超過しているかどうかをチェックします。このような例はファインチューニング中に切り詰められる可能性があり、データの損失を引き起こす可能性があります。

In [5]:
# 警告とトークンの数をカウントします
n_missing_system = 0  # システムメッセージが欠けている例の数
n_missing_user = 0  # ユーザーメッセージが欠けている例の数
n_messages = []  # 各例のメッセージ数のリスト
convo_lens = []  # 各例の合計トークン数のリスト
assistant_message_lens = []  # 各例のアシスタントメッセージのトークン数のリスト

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("システムメッセージが欠けている例の数:", n_missing_system)
print("ユーザーメッセージが欠けている例の数:", n_missing_user)
print_distribution(n_messages, "例ごとのメッセージ数")
print_distribution(convo_lens, "例ごとの合計トークン数")
print_distribution(assistant_message_lens, "例ごとのアシスタントメッセージのトークン数")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{4096}トークン制限を超える可能性がある例の数: {n_too_long} これらはファインチューニング中に切り詰められます")


システムメッセージが欠けている例の数: 1
ユーザーメッセージが欠けている例の数: 1

#### Distribution of 例ごとのメッセージ数:
min / max: 2, 9
mean / median: 3.8, 3.0
p5 / p95: 2.0, 6.6000000000000005

#### Distribution of 例ごとの合計トークン数:
min / max: 26, 8032
mean / median: 1648.4, 45.0
p5 / p95: 26.8, 4863.6

#### Distribution of 例ごとのアシスタントメッセージのトークン数:
min / max: 4, 8000
mean / median: 1610.2, 10.0
p5 / p95: 6.0, 4811.200000000001

4096トークン制限を超える可能性がある例の数: 1 これらはファインチューニング中に切り詰められます


## コストの見積もり

この最終セクションでは、ファインチューニングに使用されるトークンの総数を見積もり、それに基づいてコストを近似します。なお、ファインチューニングジョブの所要時間もトークン数とともに増加することに注意する価値があります。

In [6]:
# 価格設定とデフォルトのエポック数の見積もり
MAX_TOKENS_PER_EXAMPLE = 4096  # 1つの例あたりの最大トークン数

TARGET_EPOCHS = 3  # ターゲットのエポック数
MIN_TARGET_EXAMPLES = 100  # 最小のターゲット例数
MAX_TARGET_EXAMPLES = 25000  # 最大のターゲット例数
MIN_DEFAULT_EPOCHS = 1  # 最小のデフォルトエポック数
MAX_DEFAULT_EPOCHS = 25  # 最大のデフォルトエポック数

n_epochs = TARGET_EPOCHS  # エポック数の初期値
n_train_examples = len(dataset)  # データセットの訓練例数
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

# トレーニング中に課金されるトークン数を計算
n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"データセットにはトレーニング中に課金されるトークンが約{n_billing_tokens_in_dataset}個含まれています")
print(f"デフォルトでは、このデータセットで{n_epochs}エポックのトレーニングが行われます")
print(f"デフォルトでは、約{n_epochs * n_billing_tokens_in_dataset}個のトークンに対して課金されます")


データセットにはトレーニング中に課金されるトークンが約4306個含まれています
デフォルトでは、このデータセットで20エポックのトレーニングが行われます
デフォルトでは、約86120個のトークンに対して課金されます


See https://openai.com/pricing to estimate total costs.